In [1]:
import tensorflow as tf
import tensorflow.compat.v1.keras.backend as K
import numpy as np
import pandas as pd
from matplotlib.pyplot import imshow, legend, show
from keras_lr_finder import LRFinder  

print(tf.version.VERSION)
# tf.compat.v1.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()

2.4.1


In [2]:
lesions = {0: "Basal Cell Carcinoma", 
    1: "Lentigo",
    2: "Malignant Melanoma", 
    3: "Melanocytic naevus",
    4: "seborrhoeic keratosis",
    5: "Wart", 
    6: "Actinic Keratosis",
    7: "Squamous Cell Carcinoma",
    8: "Intraepithelial Carcinoma", 
    9: "Pyogenic Granuloma",
    10: "Haemangioma",
    11: "Dermatofibroma"
}

In [3]:
src_path_train = "dataset-split/train/"

train_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(
    directory=src_path_train,
    subset='training',
    class_mode="sparse",
    color_mode="rgb",
    batch_size=22,
    target_size=(224, 224),
    shuffle=True,
    seed=42
)

Found 7050 images belonging to 12 classes.


In [4]:
src_path_val = "dataset-split/val/"
val_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(
    directory=src_path_val,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=22,
    class_mode="sparse",
    shuffle=True,
    seed=42
)

Found 6570 images belonging to 12 classes.


In [5]:
# src_path_test = "dataset-split/test/"
# test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(
#     directory=src_path_test,
#     target_size=(224, 224),
#     color_mode="rgb",
#     batch_size=1,
#     class_mode=None,
#     shuffle=False,
#     seed=42
# )

In [6]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     for gpu in gpus:
#         tf.config.experimental.set_memory_growth(gpu, True)

In [7]:
# imshow(tf.cast(images_val[5], tf.uint8))

In [8]:
model = tf.compat.v1.keras.applications.ResNet152(weights=None)
opt = tf.compat.v1.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# model.summary()
# lr_finder = LRFinder(model)
# lr_finder.find(x, y, start_lr=0.006, end_lr=1, batch_size=28, epochs=25)
history = model.fit(train_generator,
                    validation_data = val_generator,
                    epochs=25)

Epoch 1/25
321/321 [==============================] - ETA: 0s - batch: 160.0000 - size: 21.9626 - loss: 1.9976 - accuracy: 0.3794

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


321/321 [==============================] - 280s 780ms/step - batch: 160.0000 - size: 21.9626 - loss: 1.9976 - accuracy: 0.3794 - val_loss: 2.5736 - val_accuracy: 0.1795
Epoch 2/25
321/321 [==============================] - 249s 777ms/step - batch: 160.0000 - size: 21.9626 - loss: 1.6996 - accuracy: 0.4275 - val_loss: 34.4162 - val_accuracy: 0.1181
Epoch 3/25
321/321 [==============================] - 241s 752ms/step - batch: 160.0000 - size: 21.9626 - loss: 1.6086 - accuracy: 0.4386 - val_loss: 8.5861 - val_accuracy: 0.2540
Epoch 4/25
321/321 [==============================] - 234s 731ms/step - batch: 160.0000 - size: 21.9626 - loss: 1.3546 - accuracy: 0.5218 - val_loss: 3.6068 - val_accuracy: 0.2895
Epoch 5/25
321/321 [==============================] - 234s 731ms/step - batch: 160.0000 - size: 21.9626 - loss: 1.1670 - accuracy: 0.5834 - val_loss: 3.5687 - val_accuracy: 0.3251
Epoch 6/25
321/321 [==============================] - 234s 730ms/step - batch: 160.0000 - size: 21.9626 - loss

In [9]:
# model.save('model/tensorflow-model')

In [10]:
history.history

{'loss': [1.9981704571906556,
  1.7006645143962076,
  1.607684321352776,
  1.354637801478095,
  1.1663636042716656,
  1.038291817379336,
  0.9017994484072881,
  0.7804885273791374,
  0.6807081644289883,
  0.5816305228919848,
  0.49533016988571654,
  0.41722924385933163,
  0.3700392390084816,
  0.3295619958121303,
  0.2825332507221623,
  0.2715041832779105,
  0.22769134390597226,
  0.2629429529390648,
  0.17328888544257967,
  0.17180655275610215,
  0.14446739064668246,
  0.15687182331385077,
  0.10885425984410319,
  0.1660754621622701,
  0.12121025660392293],
 'accuracy': [0.37943262,
  0.42751774,
  0.43858156,
  0.52184397,
  0.58340424,
  0.6300709,
  0.673617,
  0.7219858,
  0.7544681,
  0.792766,
  0.8286525,
  0.8560284,
  0.87078017,
  0.88851064,
  0.9039716,
  0.90836877,
  0.9238298,
  0.9104965,
  0.9469504,
  0.9417021,
  0.9534752,
  0.94397163,
  0.96326244,
  0.94851065,
  0.96],
 'val_loss': [2.5735504100952658,
  34.41617821690231,
  8.586055522778361,
  3.6067781225096

In [11]:
model.metrics_names

['loss', 'accuracy']

In [12]:
# model.save_weights('model/tensorflow-model-weights')

In [13]:
try:
    import shap
except Exception as error:
    !pip install {str(error)[17:-1]}
    import shap
except:
    print("Pacote não encontrado")

In [14]:
from PIL import Image
from numpy import asarray
import pandas as pd
test = pd.read_csv('test.csv',index_col=0)
print(test.head())
df_test = pd.DataFrame(test)
test_dataset_filenames = df_test['id'].values.tolist()
test_dataset_filenames = test_dataset_filenames[:5]

numpy_images = []
for image in test_dataset_filenames:
    print(image)
    image = Image.open(image)
    image = asarray(image)
    numpy_images.append(image)
    
x = asarray(numpy_images)

                                                  id  label
0  dataset-split/test/Wart/Wart_original_51.jpg_3...      5
1  dataset-split/test/Wart/Wart_original_74.jpg_c...      5
2  dataset-split/test/Wart/Wart_original_91.jpg_a...      5
3  dataset-split/test/Wart/Wart_original_80.jpg_0...      5
4  dataset-split/test/Wart/Wart_original_91.jpg_6...      5
dataset-split/test/Wart/Wart_original_51.jpg_32713ec0-0bd9-40bb-af39-a90afdee1428.jpg
dataset-split/test/Wart/Wart_original_74.jpg_c0649757-4964-45a5-84c8-2abc3b8fbc6e.jpg
dataset-split/test/Wart/Wart_original_91.jpg_a6f20c76-d9a6-425d-9e21-09e173c84596.jpg
dataset-split/test/Wart/Wart_original_80.jpg_07273f60-d5b8-4020-adcf-20d9665557ca.jpg
dataset-split/test/Wart/Wart_original_91.jpg_6a6dd127-6933-4137-8c81-110f1162b287.jpg


In [ ]:
print(x.shape[0])
background = x[np.random.choice(x.shape[0], 100)]
print(background)

# explain predictions of the model on four images
tf.compat.v1.disable_v2_behavior()
e = shap.DeepExplainer(model, background)
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
shap_values = e.shap_values(x)

# plot the feature attributions
shap.image_plot(shap_values, -x)

5
[[[[201 199 210]
   [215 213 224]
   [223 221 232]
   ...
   [240 238 249]
   [225 223 234]
   [212 210 221]]

  [[233 231 242]
   [238 236 247]
   [234 232 243]
   ...
   [228 226 237]
   [224 222 233]
   [223 221 232]]

  [[228 226 237]
   [233 231 242]
   [231 229 240]
   ...
   [228 226 237]
   [230 228 239]
   [233 231 242]]

  ...

  [[201 160 132]
   [203 162 134]
   [207 166 138]
   ...
   [180 130 105]
   [178 128 103]
   [179 129 104]]

  [[210 169 141]
   [210 169 141]
   [214 173 145]
   ...
   [172 122  97]
   [179 129 104]
   [193 143 118]]

  [[202 161 133]
   [200 159 131]
   [203 162 134]
   ...
   [174 124  99]
   [173 123  98]
   [175 125 100]]]


 [[[179 177 188]
   [180 178 189]
   [186 184 195]
   ...
   [175 173 184]
   [193 191 202]
   [198 196 207]]

  [[186 184 195]
   [187 185 196]
   [193 191 202]
   ...
   [177 175 186]
   [196 194 205]
   [200 198 209]]

  [[194 192 203]
   [196 194 205]
   [200 198 209]
   ...
   [182 180 191]
   [201 199 210]
   [205 2

keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode. See PR #1483 for discussion.
